<a href="https://colab.research.google.com/github/maximsachs/SEN163A_Assignment1/blob/main/SEN163A_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1


In [3]:
!wget https://raw.githubusercontent.com/maximsachs/SEN163A_Assignment1/main/transactions_test.csv

--2021-02-16 11:00:40--  https://raw.githubusercontent.com/maximsachs/SEN163A_Assignment1/main/transactions_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752509 (735K) [text/plain]
Saving to: ‘transactions_test.csv’

transactions_test.c 100%[===================>] 734.87K  --.-KB/s    in 0.06s   

2021-02-16 11:00:40 (11.8 MB/s) - ‘transactions_test.csv’ saved [752509/752509]



In [4]:
import pandas 


In [5]:
# Opening the sample data file:
df = pandas.read_csv("transactions_test.csv")
df

,timestamp,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
0,1,TRANSFER,0.01,C1231006815,170136.0,170135.99,C52983754,0.01,0.02
1,1,TRANSFER,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,9839.63
2,1,TRANSFER,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,1864.28
3,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,181.00
4,1,TRANSFER,181.00,C840083671,181.0,0.00,C38997010,21182.00,21363.00
...,...,...,...,...,...,...,...,...,...
9994,7,TRANSFER,0.01,C1086926330,47443.0,47442.99,C52983754,17.41,17.42
9995,7,TRANSFER,131687.79,C1086926330,47443.0,-84244.79,C919978702,12469.00,144156.78
9996,7,TRANSFER,14182.40,C381096925,52038.0,37855.60,M933099011,0.00,14182.40
9997,7,TRANSFER,5997.84,C1492867983,37855.6,31857.76,M1372855191,0.00,5997.84
